## Creating a mesh

We can also use the Splatter wrapper class to take an existing nerfstudio model and create a mesh!
1. **mesh:** creates a mesh via TSDF fusion

2. **query_mesh:** uses the trained model to query the mesh and returns a similarity map

3. **plot_mesh:** enables plotting of mesh features



In [1]:
import os
import sys
from pathlib import Path

import pyvista as pv

from collab_splats.wrapper import Splatter, SplatterConfig

pv.start_xvfb()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/opt/conda/envs/nerfstudio/lib/python3.10/site-packages/pyvista/plotting/utilities/xvfb.py:48: PyVistaDeprecationWarning: This function is deprecated and will be removed in future version of PyVista. Use vtk-osmesa instead.
  warnings.warn(


Set paths to the file for running splats

In [2]:
base_dir = Path("/workspace/fieldwork-data/")
session_dir = base_dir / "birds/2024-02-06/SplatsSD"

# Make the configuration
splatter_config = SplatterConfig(
    file_path=session_dir / "C0043.MP4",
    method="rade-features",
    frame_proportion=0.25,  # Use 25% of the frames within the video (or default to minimum 300 frames)
)

# Initialize the Splatter class
splatter = Splatter(splatter_config)

# Call these to populate the splatter with paths (probably a better way to do this --> maybe save out config)
splatter.preprocess()
splatter.extract_features()

transforms.json already exists at /workspace/fieldwork-data/birds/2024-02-06/environment/C0043/preproc/transforms.json
To rerun preprocessing, set overwrite=True
Output already exists for rade-features
To rerun feature extraction, set overwrite=True


### Create a mesh

We can create a mesh by calling the ```mesh()``` method. Under the hood, this runs TSDF fusion creating an integrated volume. 

In [3]:
mesher_kwargs = {
    "depth_name": "median_depth",
    "depth_trunc": 1.0,  # Should be between 1.0 and 3.0
    "voxel_size": 0.005,
    "normals_name": "normals",
    "features_name": "distill_features",
    "sdf_trunc": 0.03,
    "clean_repair": True,
    "align_floor": True,
}

splatter.mesh(
    mesher_type="Open3DTSDFFusion",
    mesher_kwargs=mesher_kwargs,
    # overwrite=True
)


Available runs:
[0] 2025-07-25_040743


### Plot the mesh!

We can use the splatter function ```plot_mesh``` to visualize given attributes of the mesh. The inherent attributes are RGB and Normals

In [4]:
splatter.plot_mesh(attribute="RGB")

Number of points: 220146
Number of cells: 427851
Bounds: BoundsTuple(x_min=-0.9965442419052124, x_max=0.6880755424499512, y_min=-0.26933255791664124, y_max=1.3843693733215332, z_min=-0.3129904866218567, z_max=0.5518640279769897)


Widget(value='<iframe src="http://localhost:38147/index.html?ui=P_0x798c4c2b3e50_0&reconnect=auto" class="pyvi…

wslink is not expecting text message:
> No PONG received after 15.0 seconds
wslink is not expecting text message:
> No PONG received after 15.0 seconds
wslink is not expecting text message:
> No PONG received after 15.0 seconds
wslink is not expecting text message:
> No PONG received after 15.0 seconds
wslink is not expecting text message:
> No PONG received after 15.0 seconds


### Using semantic queries 

The mesh contains semantic features which we can query via positive and negative prompts. The goal of this is to find points that are more similar to the positive prompts compared to the negative prompts

In [17]:
similarity = splatter.query_mesh(
    positive_queries=["feeder"], negative_queries=["ground", "leaves", "rocks"], output_fn="query-feeder.ply"
)

Plot similarity maps

In [18]:
splatter.plot_mesh(attribute=similarity, rgb=False)

Number of points: 220146
Number of cells: 427851
Bounds: BoundsTuple(x_min=-0.9965442419052124, x_max=0.6880755424499512, y_min=-0.26933255791664124, y_max=1.3843693733215332, z_min=-0.3129904866218567, z_max=0.5518640279769897)


Widget(value='<iframe src="http://localhost:41907/index.html?ui=P_0x7202e4e0f280_7&reconnect=auto" class="pyvi…

#### Semantic clustering

In [22]:
import numpy as np
import open3d as o3d

from collab_splats.utils.mesh import mesh_clustering

# Load our query mesh
mesh_dir = splatter.config["mesh_info"]["mesh"].parent
query_mesh = mesh_dir / "query-feeder.ply"
mesh = o3d.io.read_triangle_mesh(query_mesh)
similarity = np.asarray(mesh.vertex_colors)[:, 0]

# Cluster based on similarity values
clusters = mesh_clustering(
    mesh=mesh,
    similarity_values=similarity,
    similarity_threshold=0.9,
    spatial_radius=0.01,
)

Building adjacency matrix: 1333it [00:00, 40319.96it/s]


Select the largest cluster

In [23]:
# Get sizes of each cluster
cluster_sizes = [x.shape[0] for x in clusters]
cluster_size_idxs = np.argsort(cluster_sizes)[::-1]

# Pick largest cluster
target_cluster = clusters[cluster_size_idxs[0]]
cluster_vertices = np.array(target_cluster)  # Convert to numpy array

# Color the mesh to highlight the cluster
colors = np.zeros((len(mesh.vertices), 3))  # Gray background
colors[cluster_vertices] = [1, 0, 0]  # Red for selected cluster

Copy from original mesh and set properties

In [24]:
import copy

cluster_mesh = copy.deepcopy(mesh)
cluster_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

o3d.io.write_triangle_mesh(mesh_dir / "query-feeder_top-cluster.ply", cluster_mesh)

True

Plot the top cluster

In [ ]:
splatter.plot_mesh(colors)